## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [1]:
# imports
import pandas as pd
import json
from pandas.io.json import json_normalize

In [2]:
# load data into dataframe
json_df = pd.read_json('data/world_bank_projects.json')

## 1. Find the 10 countries with most projects

**Questions**
- Which column(s) identify the country/countries?
- Can one record include more than one country?
- Does each record correspond to one and only one project? (can I simply count the records/rows to get a count of projects?)
    - *Yes, both `_id` and `id` are unique project IDs*

**Plan**
1. Explore dataframe to identify country column(s)
2. Determine unique ID for country
    - May involve counting nested fields (normalize?)
3. Determine unique ID for project
4. Use something like: `df.groupby('country').value_counts()` to get count
5. Take top 10, save as list

In [3]:
json_df.columns

Index(['_id', 'approvalfy', 'board_approval_month', 'boardapprovaldate',
       'borrower', 'closingdate', 'country_namecode', 'countrycode',
       'countryname', 'countryshortname', 'docty', 'envassesmentcategorycode',
       'grantamt', 'ibrdcommamt', 'id', 'idacommamt', 'impagency',
       'lendinginstr', 'lendinginstrtype', 'lendprojectcost',
       'majorsector_percent', 'mjsector_namecode', 'mjtheme',
       'mjtheme_namecode', 'mjthemecode', 'prodline', 'prodlinetext',
       'productlinetype', 'project_abstract', 'project_name', 'projectdocs',
       'projectfinancialtype', 'projectstatusdisplay', 'regionname', 'sector',
       'sector1', 'sector2', 'sector3', 'sector4', 'sector_namecode',
       'sectorcode', 'source', 'status', 'supplementprojectflg', 'theme1',
       'theme_namecode', 'themecode', 'totalamt', 'totalcommamt', 'url'],
      dtype='object')

In [4]:
# Show all rows/columns in output
#pd.options.display.max_columns = None
#pd.options.display.max_rows = None

# This really slows things down, need to figure out cleaner way to examine df

In [5]:
json_df.head()

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Percent': 100, 'Name': 'Education for all'}","[{'code': '65', 'name': 'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Percent': 30, 'Name': 'Other economic manage...","[{'code': '24', 'name': 'Other economic manage...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en
2,{'$oid': '52b213b38594d8a2be17c782'},2014,November,2013-11-01T00:00:00Z,MINISTRY OF FINANCE AND ECONOMIC DEVEL,NaN,Tuvalu!$!TV,TV,Tuvalu,Tuvalu,...,TI,IBRD,Active,Y,"{'Percent': 46, 'Name': 'Regional integration'}","[{'code': '47', 'name': 'Regional integration'...","52,81,25,47",6060000,6060000,http://www.worldbank.org/projects/P145310?lang=en
3,{'$oid': '52b213b38594d8a2be17c783'},2014,October,2013-10-31T00:00:00Z,MIN. OF PLANNING AND INT'L COOPERATION,NaN,Republic of Yemen!$!RY,RY,Republic of Yemen,"Yemen, Republic of",...,JB,IBRD,Active,N,"{'Percent': 50, 'Name': 'Participation and civ...","[{'code': '57', 'name': 'Participation and civ...","59,57",0,1500000,http://www.worldbank.org/projects/P144665?lang=en
4,{'$oid': '52b213b38594d8a2be17c784'},2014,October,2013-10-31T00:00:00Z,MINISTRY OF FINANCE,2019-04-30T00:00:00Z,Kingdom of Lesotho!$!LS,LS,Kingdom of Lesotho,Lesotho,...,"FH,YW,YZ",IBRD,Active,N,"{'Percent': 30, 'Name': 'Export development an...","[{'code': '45', 'name': 'Export development an...","41,45",13100000,13100000,http://www.worldbank.org/projects/P144933/seco...


**Column Notes**
- `_id` seems like an internal data management id, not really a project ID that would appear on paper somewhere
- `borrower` and `country_x` fields seem promising for country codes
- `id` looks like a project ID
    - do a `value_counts` for this field and compare to `_id`
- `projectstatusdisplay` can be used to group Active vs Inactive (or some other code?) projects
    - how does this compare to `status` column?
- `regionname` could be useful for identifying outliers/errors in region/country coding


In [6]:
# Country Columns
country_cols = ['countrycode', 'countryshortname']

In [7]:
# convert '_id' column (dict) into a string column '$oid'
json_df = pd.concat([json_df, json_df['_id'].apply(pd.Series)], axis=1)

In [8]:
# confirm that '$oid' was added
json_df.columns

Index(['_id', 'approvalfy', 'board_approval_month', 'boardapprovaldate',
       'borrower', 'closingdate', 'country_namecode', 'countrycode',
       'countryname', 'countryshortname', 'docty', 'envassesmentcategorycode',
       'grantamt', 'ibrdcommamt', 'id', 'idacommamt', 'impagency',
       'lendinginstr', 'lendinginstrtype', 'lendprojectcost',
       'majorsector_percent', 'mjsector_namecode', 'mjtheme',
       'mjtheme_namecode', 'mjthemecode', 'prodline', 'prodlinetext',
       'productlinetype', 'project_abstract', 'project_name', 'projectdocs',
       'projectfinancialtype', 'projectstatusdisplay', 'regionname', 'sector',
       'sector1', 'sector2', 'sector3', 'sector4', 'sector_namecode',
       'sectorcode', 'source', 'status', 'supplementprojectflg', 'theme1',
       'theme_namecode', 'themecode', 'totalamt', 'totalcommamt', 'url',
       '$oid'],
      dtype='object')

In [9]:
# max of 'id' and '$oid' value counts
# tells me if there are duplicates
print('max of json_df[\'id\']: ' + str(json_df['id'].value_counts().max()))
print('max of json_df[\'$oid\']: ' + str(json_df['$oid'].value_counts().max()))

max of json_df['id']: 1
max of json_df['$oid']: 1


Appears that both `id` and `$oid` are unique counts of project

So - set `id` as index

In [10]:
json_df.set_index('id', inplace=True)

In [11]:
# examine country codes
json_df.groupby(['countrycode', 'countryshortname']).size()

countrycode  countryshortname             
1W           World                             2
3A           Africa                           11
4E           East Asia and Pacific             1
4P           Pacific Islands                   4
5M           Middle East and North Africa      4
7E           Europe and Central Asia           1
8S           South Asia                        2
AF           Afghanistan                       6
AG           Antigua and Barbuda               1
AL           Albania                           4
AM           Armenia                           8
AO           Angola                            1
AR           Argentina                         2
AZ           Azerbaijan                        4
BA           Bosnia and Herzegovina            1
BD           Bangladesh                       12
BF           Burkina Faso                      9
BI           Burundi                           4
BJ           Benin                             4
BO           Bolivia      

Values of `['1W', '3A', '4E', '4P', '5M', '7E', '8S']` in `'countrycode'` are not valid countries, they are regions
- look at `'borrower'` column for these records

In [12]:
# create df for projects with a region as countrycode
borrowers_df = json_df[json_df['countrycode'].isin(['1W', '3A', '4E', '4P', '5M', '7E', '8S'])]

In [13]:
# create dict for recoding borrower into country names
borrowers_dict = borrowers_df['borrower'].to_dict()
borrowers_dict

{'P075941': 'BURUNDI,RWANDA,TANZANIA',
 'P094183': 'MALAWI, MOZAMBIQUE, ZAMBIA',
 'P113794': 'DJIBOUTI,EGYPT,JORDAN,SUDAN,YEMEN',
 'P118213': 'UGANDA-COMOROS',
 'P120370': 'TANZANIA',
 'P123093': 'MINISTRIES OF TELECOM AND ICT',
 'P123933': 'UNIVERSITY OF QUEENSLAND',
 'P125018': 'ECOWAS',
 'P130174': 'GOVERNMENT OF NIGER',
 'P130343': nan,
 'P130694': 'NILE BASIN INITIATIVE',
 'P130888': 'OSS, IUCN, CILSS',
 'P131138': 'IUCN & WWF',
 'P132097': 'GOVERNMENT OF LEBANON & JORDAN',
 'P132268': 'PALESTINIAN AUTHORITY',
 'P132449': 'SECRETARIAT OF THE PACIFIC COMMUNITY',
 'P132686': 'DEPARTMENT OF FINANCE',
 'P133255': 'PARTICIPATING PICS',
 'P133329': 'BREASTFEEDING PROMOTION NETWORK OF INDIA',
 'P133572': 'WORLD ORGANISATION FOR ANIMAL HEALTH',
 'P144335': 'GOVERNMENT OF NEPAL',
 'P144655': 'CAWTAR',
 'P144902': 'BANK EXECUTED',
 'P145057': 'PACIFIC AVIATION SECURITY OFFICE',
 'P145160': 'GOVERNMENT OF MALI'}

In [14]:
# turn above dict into new dict with the borrower values (from above) as keys in this dict
country_update = {key: None for key in borrowers_dict.values()}
country_update

{'BANK EXECUTED': None,
 'BREASTFEEDING PROMOTION NETWORK OF INDIA': None,
 'BURUNDI,RWANDA,TANZANIA': None,
 'CAWTAR': None,
 'DEPARTMENT OF FINANCE': None,
 'DJIBOUTI,EGYPT,JORDAN,SUDAN,YEMEN': None,
 'ECOWAS': None,
 'GOVERNMENT OF LEBANON & JORDAN': None,
 'GOVERNMENT OF MALI': None,
 'GOVERNMENT OF NEPAL': None,
 'GOVERNMENT OF NIGER': None,
 'IUCN & WWF': None,
 'MALAWI, MOZAMBIQUE, ZAMBIA': None,
 'MINISTRIES OF TELECOM AND ICT': None,
 'NILE BASIN INITIATIVE': None,
 'OSS, IUCN, CILSS': None,
 'PACIFIC AVIATION SECURITY OFFICE': None,
 'PALESTINIAN AUTHORITY': None,
 'PARTICIPATING PICS': None,
 'SECRETARIAT OF THE PACIFIC COMMUNITY': None,
 'TANZANIA': None,
 'UGANDA-COMOROS': None,
 'UNIVERSITY OF QUEENSLAND': None,
 'WORLD ORGANISATION FOR ANIMAL HEALTH': None,
 nan: None}

In [16]:
# These values don't give me a country name so remove them from this recode dictionary

entriesToRemove = ['BANK EXECUTED',
                   'CAWTAR',
                   'DEPARTMENT OF FINANCE',
                   'ECOWAS',
                   'IUCN & WWF',
                   'MINISTRIES OF TELECOM AND ICT',
                   'NILE BASIN INITIATIVE',
                   'OSS, IUCN, CILSS',
                   'PACIFIC AVIATION SECURITY OFFICE',
                   'PARTICIPATING PICS',
                   'SECRETARIAT OF THE PACIFIC COMMUNITY',
                   'UNIVERSITY OF QUEENSLAND',
                   'WORLD ORGANISATION FOR ANIMAL HEALTH']
for k in entriesToRemove:
    country_update.pop(k, None)

country_update

{'BREASTFEEDING PROMOTION NETWORK OF INDIA': None,
 'BURUNDI,RWANDA,TANZANIA': None,
 'DJIBOUTI,EGYPT,JORDAN,SUDAN,YEMEN': None,
 'GOVERNMENT OF LEBANON & JORDAN': None,
 'GOVERNMENT OF MALI': None,
 'GOVERNMENT OF NEPAL': None,
 'GOVERNMENT OF NIGER': None,
 'MALAWI, MOZAMBIQUE, ZAMBIA': None,
 'PALESTINIAN AUTHORITY': None,
 'TANZANIA': None,
 'UGANDA-COMOROS': None,
 nan: None}

In [17]:
# need to get rid of the nan key
country_update = {k: country_update[k] for k in country_update if isinstance(k, str)}
country_update

{'BREASTFEEDING PROMOTION NETWORK OF INDIA': None,
 'BURUNDI,RWANDA,TANZANIA': None,
 'DJIBOUTI,EGYPT,JORDAN,SUDAN,YEMEN': None,
 'GOVERNMENT OF LEBANON & JORDAN': None,
 'GOVERNMENT OF MALI': None,
 'GOVERNMENT OF NEPAL': None,
 'GOVERNMENT OF NIGER': None,
 'MALAWI, MOZAMBIQUE, ZAMBIA': None,
 'PALESTINIAN AUTHORITY': None,
 'TANZANIA': None,
 'UGANDA-COMOROS': None}

In [18]:
# dict to update df (based on above dict)

country_recodes = {'BREASTFEEDING PROMOTION NETWORK OF INDIA': ['India'],
                   'BURUNDI,RWANDA,TANZANIA': ['Burundi', 'Rwanda', 'Tanzania'],
                   'DJIBOUTI,EGYPT,JORDAN,SUDAN,YEMEN': ['Djibouti', 'Egypt, Arab Republic of', 'Jordan', 'Sudan', 'Yemen, Republic of'],
                   'GOVERNMENT OF LEBANON & JORDAN': ['Lebanon', 'Jordan'],
                   'GOVERNMENT OF MALI': ['Mali'],
                   'GOVERNMENT OF NEPAL': ['Nepal'],
                   'GOVERNMENT OF NIGER': ['Niger'],
                   'MALAWI, MOZAMBIQUE, ZAMBIA': ['Malawi', 'Mozambique', 'Zambia'],
                   'PALESTINIAN AUTHORITY': ['West Bank and Gaza'],
                   'TANZANIA': ['Tanzania'],
                   'UGANDA-COMOROS': ['Uganda', 'Comoros']}

**TODO**
- apply these recodes to the original dataframe
- count of projects per country
    - need to account for multiple countries per project (due to recodes)
    - ffill `id` ?
- questions 2/3